In [ ]:
import nltk
nltk.download('popular')

In [ ]:
import pandas as pd
import numpy as np
import csv
import json

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data = pd.read_json("gdrive/My Drive/public/data_latih.json")#.read()

In [ ]:
data.head(3)

In [ ]:
len(data)

### Membuat dictionary untuk membuat kata menjadi baku

In [ ]:
key = pd.read_csv("gdrive/My Drive/public/key_norm.csv")
kamus = key.set_index('singkat')['hasil'].to_dict()

### Fungsi untuk membersihkan

In [ ]:
!pip install Sastrawi

In [ ]:
# Clean character
pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[A-Za-z0-9./]+'
pat3 = r'RT'
pat4 = r'#[A-Za-z0-9]+'
combined_pat = r'|'.join((pat1,pat2,pat3,pat4))
# Tokenization
from nltk import WordPunctTokenizer
tok = WordPunctTokenizer()
# Stop word
from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))
#Stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
from bs4 import BeautifulSoup
import re
def tweet_cleaner(text):
  soup = BeautifulSoup(text,'lxml')
  souped = soup.get_text()
  stripped = re.sub(combined_pat,' ',str(souped))

  try:
    clean = stripped.decode('utf-8').replace(u'\ufffd','?')
  except:
    clean = stripped

  letters_only = re.sub('[^a-zA-Z]',' ',clean)
  lower_case = letters_only.lower()
  words = tok.tokenize(lower_case)
  change_words=[]
  for w in words:
    if w not in kamus:
      change_words.append(w)
    else:
      change_words.append(kamus[w])
  stemw = " ".join(change_words).strip()
#   stemmer = factory.create_stemmer()
  stemmer1 = stemmer.stem(stemw)
  wordsnew = tok.tokenize(stemmer1)
  filtered_words = [w for w in wordsnew if not w in stop_words]
  return (' '.join(filtered_words)).strip()

In [ ]:
%%time

# print("Cleaning and parsing the tweets...\n")
clean_text =[]
for i in range(len(data)):
  clean_text.append(tweet_cleaner(data['isi'][i]))
print ("Done")

In [ ]:
clean_text

In [ ]:
data_tweet = pd.DataFrame({
    'isi': clean_text,
    'sentimen' : data['sentimen']})
data_tweet.head(5)

In [ ]:
tweet_pos = data_tweet[data_tweet.sentimen=='positif']
tweet_nega = data_tweet[data_tweet.sentimen=='negatif']
tweet_netr = data_tweet[data_tweet.sentimen=='netral']

#menggabungkan masing-masing sentimen
tweet_positif = " ".join(tweet_pos['isi'])
tweet_negatif = " ".join(tweet_nega['isi'])
tweet_netral = " ".join(tweet_netr['isi'])
print(tweet_positif)

In [ ]:
kata_pos = tok.tokenize(tweet_positif)
kata_neg = tok.tokenize(tweet_negatif)
kata_net = tok.tokenize(tweet_netral)

In [ ]:
print("Jumlah tweet positif: ", '{:4.2f}'.format(len(tweet_pos)/len(data)*100), '%', 'dengan kata dasar sebanyak: ', len(kata_pos))
print("Jumlah tweet negatif: ", '{:4.2f}'.format(len(tweet_nega)/len(data)*100), '%', 'dengan kata dasar sebanyak: ', len(kata_neg))
print("Jumlah tweet netral: ", '{:4.2f}'.format(len(tweet_netr)/len(data)*100), '%', 'dengan kata dasar sebanyak: ', len(kata_net))

In [ ]:
from matplotlib import pyplot
from wordcloud import WordCloud
from PIL import Image

In [ ]:
mask_pos = np.array(Image.open("gdrive/My Drive/public/conan1.png"))
mask_neg = np.array(Image.open("gdrive/My Drive/public/kid.png"))
mask_net = np.array(Image.open("gdrive/My Drive/public/twitter_logo.png"))

In [ ]:
wordcloud_pos = WordCloud(width=800,height=600,background_color="white",mask=mask_pos).generate(tweet_positif)
pyplot.clf()
pyplot.imshow(wordcloud_pos)
pyplot.axis('off')
pyplot.title('Positif')
#pyplot.savefig("indonesia-raya.png",dpi = (300))
pyplot.show()

In [ ]:
wordcloud_net = WordCloud(width=800,height=600,background_color="white",mask=mask_net).generate(tweet_positif)
pyplot.clf()
pyplot.imshow(wordcloud_net)
pyplot.axis('off')
pyplot.title('Netral')
#pyplot.savefig("indonesia-raya.png",dpi = (300))
pyplot.show()

In [ ]:
wordcloud_neg = WordCloud(width=800,height=600,background_color="white",collocations=False).generate(tweet_positif)
pyplot.clf()
pyplot.imshow(wordcloud_neg)
pyplot.axis('off')
pyplot.title('Negatif')
#pyplot.savefig("indonesia-raya.png",dpi = (300))
pyplot.show()

### Membagi data dengan 20 % sebagai data test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
np.random.seed(32)

In [ ]:
class_name=set(data_tweet['sentimen'])
class_name

In [ ]:
data_tweet.head(5)

In [ ]:
trainSet, testSet = train_test_split(data_tweet, test_size = 0.2, stratify = data_tweet['sentimen'], random_state=0)

In [ ]:
count_vect = CountVectorizer(analyzer="word", ngram_range=(1,1), stop_words = stopwords.words('english'), lowercase=True)

train_vect = count_vect.fit_transform(trainSet['isi'].astype(str)) #fit transform pada data train A
test_vect = count_vect.transform(testSet['isi'].astype(str)) #transform data test

### Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
train_vect

In [ ]:
AdaClassifier = AdaBoostClassifier()
# AdaClassifier = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=10.0, n_estimators=100, random_state=42)
AdaClassifier.fit(train_vect , trainSet['sentimen'])
Predict1 = AdaClassifier.predict(test_vect)
accuracy_1 = accuracy_score(Predict1, testSet['sentimen'])

In [ ]:
print ("Accuracy pada test data dengan AdaBoost:", accuracy_1)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
SVM = svm.SVC(kernel='linear')

SVM.fit(train_vect , trainSet['sentimen'])
Predict2 = SVM.predict(test_vect)
accuracy_2=accuracy_score(Predict2, testSet['sentimen'])
print ("Accuracy pada test data dengan SVM:", accuracy_2)

In [ ]:
sentimen = {'negatif': '0','positif': '1','netral': '2'} #ubah sentiment menjadi angka
trainSet.sentimen = [sentimen[item] for item in trainSet.sentimen]
testSet.sentimen = [sentimen[item] for item in testSet.sentimen]

In [ ]:
trainSet.head()

In [ ]:
testSet.head()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(train_vect , trainSet['sentimen'])
Predict3 = model.predict(test_vect)
r_sq = model.score(train_vect , trainSet['sentimen'])
print('Coefficient of determination (R^2):', r_sq)

In [ ]:
print ("Accuracy pada test data dengan AdaBoost:", accuracy_1)
print ("Accuracy pada test data dengan SVM:", accuracy_2)
print('Coefficient Regresi Linier (R^2):', r_sq)

### Confussion Matrix

In [ ]:
sentimen_balik = {'0' : 'negatif','1' : 'positif', '2': 'netral'} #ubah sentiment menjadi angka
trainSet.sentimen = [sentimen_balik[item] for item in trainSet.sentimen]
testSet.sentimen = [sentimen_balik[item] for item in testSet.sentimen]

In [ ]:
class_name=set(data_tweet['sentimen']) #mendefinisikan class dari data

In [ ]:
class_name

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(testSet["sentimen"] , Predict1, classes=class_name,
                      title='Confusion matrix, AdaBoost')
plt.show()

In [ ]:
# Plot non-normalized confusion matrix
plot_confusion_matrix(testSet["sentimen"] , Predict2, classes=class_name,
                      title='Confusion matrix, SVM')
plt.show()